# Purpose

2022-07-05: Run **subreddit-level** clustering jobs in parallel with `hydra`.



# Imports & Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
import logging
import os
from pathlib import Path

import numpy as np
import pandas as pd
import plotly
import seaborn as sns

import mlflow
import hydra

import subclu
from subclu.eda.aggregates import compare_raw_v_weighted_language
from subclu.utils import set_working_directory, get_project_subfolder
from subclu.utils.eda import (
    setup_logging, counts_describe, value_counts_and_pcts,
    notebook_display_config, print_lib_versions,
    style_df_numeric, reorder_array,
)
from subclu.utils.mlflow_logger import MlflowLogger
from subclu.utils.hydra_config_loader import LoadHydraConfig
from subclu.utils.data_irl_style import (
    get_colormap, theme_dirl, 
    get_color_dict, base_colors_for_manual_labels,
    check_colors_used,
)
from subclu.data.data_loaders import LoadPosts, LoadSubreddits, create_sub_level_aggregates


# ===
# imports specific to this notebook
from collections import Counter
# import umap
# import openTSNE
# from openTSNE import TSNE

# import hdbscan

import sklearn
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize  # if we normalize the data, euclidean distance is approx of cosine

from sklearn.cluster import KMeans, DBSCAN, OPTICS, AgglomerativeClustering

from subclu.models.clustering import ClusterEmbeddings
from subclu.models.clustering_utils import plot_elbow_and_get_k


print_lib_versions([hydra, np, pd, plotly, sklearn, sns, subclu])

python		v 3.7.10
===
hydra		v: 1.1.0
numpy		v: 1.19.5
pandas		v: 1.2.4
plotly		v: 4.14.3
sklearn		v: 0.24.1
seaborn		v: 0.11.1
subclu		v: 0.5.0


In [3]:
# plotting
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
plt.style.use('default')

setup_logging()
notebook_display_config()

# Set sqlite database as MLflow URI

In [4]:
# use new class to initialize mlflow
mlf = MlflowLogger(tracking_uri='sqlite')
mlflow.get_tracking_uri()

'sqlite:////home/jupyter/subreddit_clustering_i18n/mlflow_sync/djb-100-2021-04-28-djb-eda-german-subs/mlruns.db'

## Get list of experiments with new function

In [5]:
mlf.list_experiment_meta(output_format='pandas').tail(9)

,experiment_id,name,artifact_location,lifecycle_stage
25,25,v0.4.1_mUSE_clustering_new_metrics,gs://i18n-subreddit-clustering/mlflow/mlruns/25,active
26,26,v0.4.1_nearest_neighbors_test,gs://i18n-subreddit-clustering/mlflow/mlruns/26,active
27,27,v0.4.1_nearest_neighbors,gs://i18n-subreddit-clustering/mlflow/mlruns/27,active
28,28,v0.5.0_mUSE_aggregates_test,gs://i18n-subreddit-clustering/mlflow/mlruns/28,active
29,29,v0.5.0_mUSE_aggregates,gs://i18n-subreddit-clustering/mlflow/mlruns/29,active
30,30,v0.5.0_mUSE_clustering_test,gs://i18n-subreddit-clustering/mlflow/mlruns/30,active
31,31,v0.5.0_mUSE_clustering,gs://i18n-subreddit-clustering/mlflow/mlruns/31,active
32,32,v0.5.0_nearest_neighbors_test,gs://i18n-subreddit-clustering/mlflow/mlruns/32,active
33,33,v0.5.0_nearest_neighbors,gs://i18n-subreddit-clustering/mlflow/mlruns/33,active


## Get experiment IDs to use for clustering

There is only one run that completed and covers all subreddits (the other run ran an a subset).
Let's use:<br>
`bfe6cbd59a21480c8c2b9923a3a9cbd6`

In [8]:
%%time

df_mlf = mlf.search_all_runs(experiment_ids=[29])
df_mlf.shape

CPU times: user 48.6 ms, sys: 8.2 ms, total: 56.8 ms
Wall time: 56.1 ms


(2, 36)

In [9]:
df_mlf.head()

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.time_fxn-df_posts_agg_c1,metrics.time_fxn-full_aggregation_fxn_minutes,metrics.df_v_subs-rows,metrics.memory_free,metrics.df_subs_agg_c1-rows,metrics.df_posts_agg_c1-rows,metrics.df_v_subs-cols,metrics.time_fxn-data_loading_time,metrics.df_posts_agg_c1-cols,metrics.df_subs_agg_c1-cols,metrics.time_fxn-df_subs_agg_c1,metrics.memory_used,metrics.memory_total,metrics.cpu_count,metrics.df_v_post_comments-rows,metrics.df_v_post_comments-cols,metrics.memory_used_percent,params.weight_post_and_comments,params.weight_subreddit_meta,params.host_name,params.memory_total,params.embeddings_bucket,params.cpu_count,params.embeddings_subreddit_path,params.embeddings_post_and_comments_path,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.source.name,tags.host_name,tags.mlflow.source.git.commit
0,bfe6cbd59a21480c8c2b9923a3a9cbd6,29,FINISHED,gs://i18n-subreddit-clustering/mlflow/mlruns/29/bfe6cbd59a21480c8c2b9923a3a9cbd6/artifacts,2022-07-01 11:42:15.973000+00:00,2022-07-01 14:36:59.617000+00:00,161.522672,174.728815,196371.0,1381009.0,81973.0,16360314.0,514.0,0.926008,515.0,514.0,1.536616,166480.0,1444961.0,96.0,16360314.0,515.0,0.115214,0.85,0.15,djb-100-2021-04-28-djb-eda-german-subs,1444961,i18n-subreddit-clustering,96,i18n_topic_model_batch/runs/20220629/subreddits/text/embedding/2022-06-29_084555,i18n_topic_model_batch/runs/20220629/post_and_comment_text_combined/text_subreddit_seeds/embedding/2022-06-29_091925,jupyter,LOCAL,/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py,djb-100-2021-04-28-djb-eda-german-subs,4fbcc02af1efc611f6dc25f1ac5369375346aa8c
1,03b1c15c736340cc8d33ce75debdc7f7,29,FINISHED,gs://i18n-subreddit-clustering/mlflow/mlruns/29/03b1c15c736340cc8d33ce75debdc7f7/artifacts,2022-07-01 11:38:17.732000+00:00,2022-07-01 11:41:31.509000+00:00,1.128882,3.209811,196371.0,1381402.0,573.0,123000.0,514.0,0.953466,515.0,514.0,0.008693,110789.0,1444961.0,96.0,16360314.0,515.0,0.076673,0.85,0.15,djb-100-2021-04-28-djb-eda-german-subs,1444961,i18n-subreddit-clustering,96,i18n_topic_model_batch/runs/20220629/subreddits/text/embedding/2022-06-29_084555,i18n_topic_model_batch/runs/20220629/post_and_comment_text_combined/text_subreddit_seeds/embedding/2022-06-29_091925,jupyter,LOCAL,/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py,djb-100-2021-04-28-djb-eda-german-subs,4fbcc02af1efc611f6dc25f1ac5369375346aa8c


In [8]:
# mask_finished = df_mlf['status'] == 'FINISHED'
# mask_df_similarity_complete = ~df_mlf['metrics.df_sub_level_agg_a_post_only_similarity-rows'].isnull()

# df_mlf_clustering_candidates = df_mlf[mask_finished & mask_df_similarity_complete]
# df_mlf_clustering_candidates.shape

In [9]:
# cols_with_multiple_vals = df_mlf_clustering_candidates.columns[df_mlf_clustering_candidates.nunique(dropna=False) > 1]

# df_mlf_clustering_candidates[cols_with_multiple_vals]

# Inspect config for clustering job

This config should include:
- data to load for clustering
- parameters for clustering algo
- hydra overrides to run jobs in parallel

In [10]:
test_experiment = 'v0.5.0_mUSE_clustering_test'

cfg_cluster_test = LoadHydraConfig(
    config_name='clustering_v0.4.1_subreddit_base.yaml',
    config_path="../config",
    overrides=[
        f"mlflow_experiment_name={test_experiment}"
#         f"data_text_and_metadata=top_subreddits_2021_07_16",
#         f"data_embeddings_to_cluster=top_subs-2021_07_16-use_multi_lower_case_false_00",
    ],
)

print([k for k in cfg_cluster_test.config_dict.keys()])

['data_text_and_metadata', 'data_embeddings_to_cluster', 'clustering_algo', 'embeddings_to_cluster', 'n_sample_embedding_rows', 'n_max_clusters_to_check_for_optimal_k', 'filter_embeddings', 'mlflow_tracking_uri', 'mlflow_experiment_name', 'pipeline_config']


In [11]:
cfg_cluster_test.config_dict['n_max_clusters_to_check_for_optimal_k']

4000

In [13]:
# data with embeddings
cfg_cluster_test.config_dict['data_embeddings_to_cluster']

{'config_description': 'Config for embeddings that at post level included OCR text + URL stems',
 'run_uuid': '4144c443722e47ef9cd2f42a5e813f3b',
 'l_ix_sub': ['subreddit_name', 'subreddit_id'],
 'l_ix_post': ['subreddit_name', 'post_id'],
 'df_post_level_agg_b_post_and_comments': None,
 'df_post_level_agg_c_post_comments_sub_desc': 'df_post_level_agg_c_post_comments_sub_desc',
 'df_sub_level_agg_a_post_only': 'df_sub_level_agg_a_post_only',
 'df_sub_level_agg_a_post_only_similarity': None,
 'df_sub_level_agg_a_post_only_similarity_pair': None,
 'df_sub_level_agg_a_post_only_similarity_top_pair': None,
 'df_sub_level_agg_b_post_and_comments': None,
 'df_sub_level_agg_b_post_and_comments_similarity': None,
 'df_sub_level_agg_b_post_and_comments_similarity_pair': None,
 'df_sub_level_agg_c_post_comments_and_sub_desc': 'df_sub_level_agg_c_post_comments_and_sub_desc',
 'df_sub_level_agg_c_post_comments_and_sub_desc_similarity': None,
 'df_sub_level_agg_c_post_comments_and_sub_desc_similari

In [14]:
# clustering algo
# cfg_cluster_test.config_dict['clustering_algo']

# Run as python fnx

In [ ]:
%%time

# use ** to over-ride config defaults
cls_test = ClusterEmbeddings(
    **{
        **cfg_cluster_test.config_dict,
        **{
            'n_sample_embedding_rows': 5000,
            'mlflow_experiment_name': test_experiment,
        }
    }
)

cls_test.run_clustering()

# Run commmand line fxn

The clustering fxn is in `subclu.models.conlustering.py`

Notes:
- We need to use the `-m` flag to run as a submodule (and allow relative imports)
- When using the `-m` flag, REMOVE the `.py` ending of the file!
- In the command line for hydra we can override w/o having to use the `+` sign.
    - https://hydra.cc/docs/tutorials/basic/your_first_app/config_file

## Check & set paths

In [15]:
test_experiment

'v0.4.1_mUSE_clustering_test'

In [16]:
os.getcwd()

'/home/jupyter/subreddit_clustering_i18n/notebooks/v0.4.1'

In [17]:
path_djb_repo = '/home/david.bermejo/repos/subreddit_clustering_i18n/' 
path_djb_models = '/home/david.bermejo/repos/subreddit_clustering_i18n/subclu/models' 
file_clustering_py = 'subclu.models.clustering'

config_name = 'clustering_v0.4.1_subreddit_base'

## Run clustering from CLI

```bash
!cd $path_djb_repo && python -m $file_clustering_py \
    mlflow_experiment_name=$test_experiment \
    n_sample_embedding_rows=4000 \
    filter_embeddings.filter_subreddits.minimum_column_value=9
```

In [18]:
file_clustering_py

'subclu.models.clustering'

In [19]:
config_name

'clustering_v0.4.1_subreddit_base'

In [20]:
# run on sample data, test experiment

!cd $path_djb_repo && python -m $file_clustering_py \
    --config-name $config_name \
    mlflow_experiment_name=$test_experiment \
    n_sample_embedding_rows=5000 \
    filter_embeddings.filter_subreddits.minimum_column_value=9

CFG keys: dict_keys(['data_text_and_metadata', 'data_embeddings_to_cluster', 'clustering_algo', 'embeddings_to_cluster', 'n_sample_embedding_rows', 'n_max_clusters_to_check_for_optimal_k', 'filter_embeddings', 'mlflow_tracking_uri', 'mlflow_experiment_name', 'pipeline_config'])
`2022-01-20 16:28:13,995` | `INFO` | `Define cluster class...`
`2022-01-20 16:28:14,861` | `INFO` | `== Start run_aggregation() method ==`
`2022-01-20 16:28:14,861` | `INFO` | `MLflow tracking URI: sqlite:////home/jupyter/subreddit_clustering_i18n/mlflow_sync/djb-100-2021-04-28-djb-eda-german-subs/mlruns.db`
`2022-01-20 16:28:15,030` | `INFO` | `=== START CLUSTERING - Process ID 9109`
`2022-01-20 16:28:15,125` | `INFO` | `host_name: djb-100-2021-04-28-djb-eda-german-subs`
`2022-01-20 16:28:15,125` | `INFO` | `cpu_count: 80`
`2022-01-20 16:28:15,198` | `INFO` | `RAM stats:
{'memory_used_percent': '0.42%', 'memory_total': '524,059', 'memory_used': '2,199', 'memory_free': '520,546'}`
`2022-01-20 16:28:15,309` | `IN

In [22]:
# Since we're including 50k subreddits for modeling
# if clusters were even, at k=1000, we'd expect an average of 50 subreddits per cluster
50000 / 1000

50.0

In [23]:
# We might need to go down to 2,000 >= clusters to get more fine-grained sub-clusters
50000 / 2000

25.0

In [24]:
# We might need to go down to 4,000 clusters to get more fine-grained sub-clusters
50000 / 4000

12.5

## Test multirun

Jobs can run in parallel with `--multirun` flag!

In [25]:
print(fr"""
cd {path_djb_repo} && python -m {file_clustering_py} --multirun \
    "filter_embeddings.filter_subreddits.minimum_column_value=range(2, 10, 2)" \
    mlflow_experiment_name={test_experiment} \
    n_sample_embedding_rows=4200
""")


cd /home/david.bermejo/repos/subreddit_clustering_i18n/ && python -m subclu.models.clustering --multirun \
    "filter_embeddings.filter_subreddits.minimum_column_value=range(2, 10, 2)" \
    mlflow_experiment_name=v0.4.1_mUSE_clustering_test \
    n_sample_embedding_rows=4200



### Example: 7 jobs, 5 at the same time

With this command:

```bash
!cd $path_djb_repo && python -m $file_clustering_py --multirun \
    "filter_embeddings.filter_subreddits.minimum_column_value=range(2, 9, 1)" \
    mlflow_experiment_name=$test_experiment \
    n_sample_embedding_rows=4200
```

We can adjust `n_jobs` (how many in parallel) in the root config:
```yaml
# config: config/clustering_v0.4.1_subreddit_base.yaml

# Change n_jobs(parallel jobs) & logging for hydra itself
hydra:
  launcher:
    # override the number of jobs for joblib
    n_jobs: 5
```

With n_job=5, we expect:
- 7 jobs total
    - because we'll change the `filter_embeddings.filter_subreddits.minimum_column_value` parameter to:
        - Expanded range = [2, 3, 4, 5, 6, 7, 8]
- 2 batches of jobs
    - Each batch = 5 jobs in parallel
        - batch 1: job # IDs: 0, 1, 2, 3, 4
        - batch 2: job # IDs: 5, 6

In [ ]:
# run on sample data, multi-run

!cd $path_djb_repo && python -m $file_clustering_py --multirun \
    "filter_embeddings.filter_subreddits.minimum_column_value=range(2, 9, 1)" \
    "hydra.launcher.n_jobs=3" \
    mlflow_experiment_name=$test_experiment \
    n_sample_embedding_rows=4200

### Results after making the `cluster_embeddings` function return nothing:

Success!!! 7 jobs ran in 2 batches without any errors!

## Run on full data (one job)

In [40]:
# # run on full data, still a test

# !cd $path_djb_repo && python -m $file_clustering_py mlflow_experiment_name=$test_experiment

In [56]:
# # # run on full data, no longer a test

# !cd $path_djb_repo && python -m $file_clustering_py

## Run on full data (multijob)

In [32]:
config_name = 'clustering_v0.4.1_subreddit_base'

In [33]:
# ward-related jobs
!cd $path_djb_repo && python -m $file_clustering_py --multirun \
    --config-name $config_name \
    "clustering_algo.model_kwargs.linkage='ward'" \
    "clustering_algo.model_kwargs.affinity='euclidean'" \
    "filter_embeddings.filter_subreddits.minimum_column_value=range(2, 11, 1)" \
    "pipeline_config.normalize.add_step=choice(false, true)" \
    "pipeline_config.reduce.add_step=choice(false, true)"

[2022-01-20 17:57:15,863][HYDRA] Joblib.Parallel(n_jobs=11,backend=loky,prefer=processes,require=None,verbose=0,timeout=None,pre_dispatch=2*n_jobs,batch_size=auto,temp_folder=None,max_nbytes=None,mmap_mode=r) is launching 36 jobs
[2022-01-20 17:57:15,863][HYDRA] Launching jobs, sweep output dir : /home/jupyter/subreddit_clustering_i18n/hydra_runs/multirun/2022-01-20/17-57-11
[2022-01-20 17:57:15,863][HYDRA] 	#0 : clustering_algo.model_kwargs.linkage='ward' clustering_algo.model_kwargs.affinity='euclidean' filter_embeddings.filter_subreddits.minimum_column_value=2 pipeline_config.normalize.add_step=False pipeline_config.reduce.add_step=False
[2022-01-20 17:57:15,863][HYDRA] 	#1 : clustering_algo.model_kwargs.linkage='ward' clustering_algo.model_kwargs.affinity='euclidean' filter_embeddings.filter_subreddits.minimum_column_value=2 pipeline_config.normalize.add_step=False pipeline_config.reduce.add_step=True
[2022-01-20 17:57:15,863][HYDRA] 	#2 : clustering_algo.model_kwargs.linkage='ward

In [34]:
# non-ward linkages
#  We set normalize.add_ste=false because that's only used for Ward,
#  where normalize+euclidean = cosine distance

!cd $path_djb_repo && python -m $file_clustering_py --multirun \
    --config-name $config_name \
    "clustering_algo.model_kwargs.linkage=choice('single', 'complete', 'average')" \
    "clustering_algo.model_kwargs.affinity=choice('cosine', 'euclidean')" \
    "filter_embeddings.filter_subreddits.minimum_column_value=range(3, 11, 1)" \
    "pipeline_config.reduce.add_step=choice(false, true)" \
    "pipeline_config.normalize.add_step=false"

[2022-01-20 18:33:57,508][HYDRA] Joblib.Parallel(n_jobs=11,backend=loky,prefer=processes,require=None,verbose=0,timeout=None,pre_dispatch=2*n_jobs,batch_size=auto,temp_folder=None,max_nbytes=None,mmap_mode=r) is launching 96 jobs
[2022-01-20 18:33:57,508][HYDRA] Launching jobs, sweep output dir : /home/jupyter/subreddit_clustering_i18n/hydra_runs/multirun/2022-01-20/18-33-45
[2022-01-20 18:33:57,509][HYDRA] 	#0 : clustering_algo.model_kwargs.linkage=single clustering_algo.model_kwargs.affinity=cosine filter_embeddings.filter_subreddits.minimum_column_value=3 pipeline_config.reduce.add_step=False pipeline_config.normalize.add_step=False
[2022-01-20 18:33:57,509][HYDRA] 	#1 : clustering_algo.model_kwargs.linkage=single clustering_algo.model_kwargs.affinity=cosine filter_embeddings.filter_subreddits.minimum_column_value=3 pipeline_config.reduce.add_step=True pipeline_config.normalize.add_step=False
[2022-01-20 18:33:57,509][HYDRA] 	#2 : clustering_algo.model_kwargs.linkage=single clusteri

# Test & fix multi-run error

It seems like the error was because we were returning a `ClusterEmbeddings` object that couldn't be pickled.

Fix: added a flag to return `None` (nothing) by default so that way there is nothing to pickle.


## Test 7 jobs, 5 at the same time

With this command:

```bash
!cd $path_djb_repo && python -m $file_clustering_py --multirun \
    "filter_embeddings.filter_subreddits.minimum_column_value=range(2, 9, 1)" \
    mlflow_experiment_name=$test_experiment \
    n_sample_embedding_rows=4200
```

We expect:
- 7 jobs total
- 5 in parallel
- 2 batches of jobs
    - batch 1: jobs 0, 1, 2, 3, 4
    - batch 2: jobs 5, 6

--- 

**Outcomes** 

Hydra:
- seems to note the 7 jobs (0 through 6)
- launches 5 in parallel for first batch (holding off on the last 2)
    - we confirm it in logs and in mlflow DB
- the first bach of 5 jobs launches and completes
- However, we still get the same error when the first job from the 2nd batch kicks off

Follow up tests/questions:
- Does it make sense to test only 5 jobs at a time?
- What if we increase n_jobs (in parallel) to 10? would all the jobs in a batch complete?

**IS THE PROBLEM Because the hydra function now returns an object??**
**YES!!!**

- What if we set it to return nothing!!!??
    - checking git blame, it looks like I've always returned the object
        - https://github.snooguts.net/david-bermejo/subreddit_clustering_i18n/blame/djb_v041_expand_coverage/subclu/models/clustering.py
    - but going to set it to return nothing anyway in case something else changed in joblib since then

```python
@hydra.main(config_path='../config', config_name="clustering_v0.4.1_subreddit_base")
def cluster_embeddings(cfg: DictConfig) -> object:
```



---

Log outputs from job that had errors:


```bash 
[2022-01-20 16:35:19,725][HYDRA] Joblib.Parallel(n_jobs=5,backend=loky,prefer=processes,require=None,verbose=0,timeout=None,pre_dispatch=2*n_jobs,batch_size=auto,temp_folder=None,max_nbytes=None,mmap_mode=r) is launching 7 jobs
[2022-01-20 16:35:19,725][HYDRA] Launching jobs, sweep output dir : /home/jupyter/subreddit_clustering_i18n/hydra_runs/multirun/2022-01-20/16-35-18
[2022-01-20 16:35:19,725][HYDRA] 	#0 : filter_embeddings.filter_subreddits.minimum_column_value=2 mlflow_experiment_name=v0.4.1_mUSE_clustering_test n_sample_embedding_rows=4200
[2022-01-20 16:35:19,725][HYDRA] 	#1 : filter_embeddings.filter_subreddits.minimum_column_value=3 mlflow_experiment_name=v0.4.1_mUSE_clustering_test n_sample_embedding_rows=4200
[2022-01-20 16:35:19,725][HYDRA] 	#2 : filter_embeddings.filter_subreddits.minimum_column_value=4 mlflow_experiment_name=v0.4.1_mUSE_clustering_test n_sample_embedding_rows=4200
[2022-01-20 16:35:19,725][HYDRA] 	#3 : filter_embeddings.filter_subreddits.minimum_column_value=5 mlflow_experiment_name=v0.4.1_mUSE_clustering_test n_sample_embedding_rows=4200
[2022-01-20 16:35:19,725][HYDRA] 	#4 : filter_embeddings.filter_subreddits.minimum_column_value=6 mlflow_experiment_name=v0.4.1_mUSE_clustering_test n_sample_embedding_rows=4200
[2022-01-20 16:35:19,725][HYDRA] 	#5 : filter_embeddings.filter_subreddits.minimum_column_value=7 mlflow_experiment_name=v0.4.1_mUSE_clustering_test n_sample_embedding_rows=4200
[2022-01-20 16:35:19,725][HYDRA] 	#6 : filter_embeddings.filter_subreddits.minimum_column_value=8 mlflow_experiment_name=v0.4.1_mUSE_clustering_test n_sample_embedding_rows=4200

...

### Jobs from 1st batch complete at slightly different intervals ###
# ===
`2022-01-20 16:36:37,768` | `INFO` | `=== END clustering full job ===`
[PosixPath('/home/jupyter/subreddit_clustering_i18n/hydra_runs/multirun/2022-01-20/16-35-18/1/clustering.log')]
`2022-01-20 16:36:37,821` | `INFO` | `=== END clustering full job ===`
`2022-01-20 16:36:38,086` | `INFO` | `=== END clustering full job ===`
`2022-01-20 16:36:38,225` | `INFO` | `  0:01:14.869412 <- Total Clustering fxn time time elapsed`
`2022-01-20 16:36:38,266` | `INFO` | `--- END clustering metrics---`
`2022-01-20 16:36:38,266` | `INFO` | `  Uploading logs to mlflow...`
`2022-01-20 16:36:38,321` | `INFO` | `RAM stats:
{'memory_used_percent': '1.90%', 'memory_used': '9,983'}`


### 1st job from 2nd batch STARTS with config ###
# ===
CFG keys: dict_keys(['data_text_and_metadata', 'data_embeddings_to_cluster', 'clustering_algo', 'embeddings_to_cluster', 'n_sample_embedding_rows', 'n_max_clusters_to_check_for_optimal_k', 'filter_embeddings', 'mlflow_tracking_uri', 'mlflow_experiment_name', 'pipeline_config'])
`2022-01-20 16:36:38,349` | `INFO` | `Define cluster class...`
[PosixPath('/home/jupyter/subreddit_clustering_i18n/hydra_runs/multirun/2022-01-20/16-35-18/0/clustering.log')]
CFG keys: dict_keys(['data_text_and_metadata', 'data_embeddings_to_cluster', 'clustering_algo', 'embeddings_to_cluster', 'n_sample_embedding_rows', 'n_max_clusters_to_check_for_optimal_k', 'filter_embeddings', 'mlflow_tracking_uri', 'mlflow_experiment_name', 'pipeline_config'])
`2022-01-20 16:36:38,449` | `INFO` | `Define cluster class...`
### Last job from 1st batch ENDS ###
# ===
`2022-01-20 16:36:38,663` | `INFO` | `=== END clustering full job ===`
### Last job from 1st batch ENDS ###
# ===
`2022-01-20 16:36:38,953` | `INFO` | `== Start run_aggregation() method ==`
`2022-01-20 16:36:38,953` | `INFO` | `MLflow tracking URI: sqlite:////home/jupyter/subreddit_clustering_i18n/mlflow_sync/djb-100-2021-04-28-djb-eda-german-subs/mlruns.db`
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/hydra/_internal/utils.py", line 211, in run_and_report
    return func()
  File "/opt/conda/lib/python3.7/site-packages/hydra/_internal/utils.py", line 379, in <lambda>
    overrides=args.overrides,
  File "/opt/conda/lib/python3.7/site-packages/hydra/_internal/hydra.py", line 139, in multirun
    ret = sweeper.sweep(arguments=task_overrides)
  File "/opt/conda/lib/python3.7/site-packages/hydra/_internal/core_plugins/basic_sweeper.py", line 157, in sweep
    results = self.launcher.launch(batch, initial_job_idx=initial_job_idx)
  File "/opt/conda/lib/python3.7/site-packages/hydra_plugins/hydra_joblib_launcher/joblib_launcher.py", line 46, in launch
    launcher=self, job_overrides=job_overrides, initial_job_idx=initial_job_idx
  File "/opt/conda/lib/python3.7/site-packages/hydra_plugins/hydra_joblib_launcher/_core.py", line 112, in launch
    for idx, overrides in enumerate(job_overrides)
  File "/opt/conda/lib/python3.7/site-packages/joblib/parallel.py", line 1054, in __call__
    self.retrieve()
  File "/opt/conda/lib/python3.7/site-packages/joblib/parallel.py", line 933, in retrieve
    self._output.extend(job.get(timeout=self.timeout))
  File "/opt/conda/lib/python3.7/site-packages/joblib/_parallel_backends.py", line 542, in wrap_future_result
    return future.result(timeout=timeout)
  File "/opt/conda/lib/python3.7/concurrent/futures/_base.py", line 435, in result
    return self.__get_result()
  File "/opt/conda/lib/python3.7/concurrent/futures/_base.py", line 384, in __get_result
    raise self._exception
TypeError: can't pickle _thread.RLock objects
```
